In [11]:
from bs4 import BeautifulSoup
import requests
import datetime
import pandas as pd
import json


month_choices = [datetime.date(2020, i, 1).strftime('%B') for i in range(1, 13)]


def format_critter_row(row):
    d = dict()
    elements = row.find_all("td")
    if not elements:
        return None
    link = elements[0].find("a")
    d["name"] = link.text.strip()
    try:
        d["page"] = link['href']
    except KeyError:
        d["page"] = ""
    d["image"] = elements[1].find("a")['href']
    d["price"] = elements[2].text.strip()
    d["location"] = elements[3].text.strip()
    d["shadow_size"] = elements[4].text.strip()
    d["time"] = elements[5].text.strip()
    d["months"] = [month for month, td in zip(month_choices, elements[6:18])
                   if td.text.strip() != "-"]

    return d


def generate_critter_rows(tab):
    for tr in tab.find_all("tr")[1:]:
        row = format_critter_row(tr)
        if row:
            yield row


def get_critter_list(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    content = soup.find("div", id="mw-content-text")
    return {tab['title']: list(generate_critter_rows(tab))
            for tab in content.find_all("div", {"class": "tabbertab"})}




wiki_base_url = "https://animalcrossing.fandom.com"
fish_url = "/".join([wiki_base_url, "wiki/Fish_(New_Horizons)"])
hemisphere = "Northern Hemisphere"

fish_lists = {key: pd.DataFrame(value) for key, value in get_critter_list(fish_url).items()}
fish_list = fish_lists[hemisphere]

In [33]:
from IPython.display import HTML, Image
this_month = datetime.date.today().month
month_list = [month_choices[int(i + this_month - 1) % 12] for i in range(12)]

def months_left(critter_months):
    month_count = 0
    for month in month_list:
        if month not in critter_months:
            return month_count
        else:
            month_count += 1
    return month_count

def months_until(critter_months):
    month_count = 0
    for month in month_list:
        if month in critter_months:
            return month_count
        else:
            month_count += 1
    return month_count

def compute_hot_list(critter_list):
    df = critter_list.copy()
    df["months_left"] = df["months"].apply(months_left)
    df["months_until"] = df["months"].apply(months_until)
    df["price"] = pd.to_numeric(df["price"])
    name_link = lambda row: f"<a target='_blank' href='{wiki_base_url}/{row['page']}'>{row['name']}</a>"
    df["name"] = df.apply(name_link, axis=1)
    df["image"] = df.apply(lambda row: f"<img src='{row['image']}' style='width:32px;height:32px;'/>", axis=1)
    del df["months"]
    del df["page"]                                     if lst[0] in ms][0])
    df = df.sort_values(by=["months_until", "months_left", "location", "price"],
                        ascending=[True, True, True, False])
    return df

pd.set_option('display.max_rows', None)
fish_hot_list = compute_hot_list(fish_list)
HTML(fish_hot_list.to_html(
    render_links=True,
    escape=False
))

,name,image,price,location,shadow_size,time,months_left,months_until
66,Blue marlin,,10000,Pier,6,All day,1,0
65,Tuna,,7000,Pier,6,All day,1,0
60,Dab,,300,Sea,3,All day,1,0
16,Loach,,400,River,2,All day,2,0
28,Golden trout,,15000,River (Clifftop),3,4 PM - 9 AM,2,0
77,Oarfish,,9000,Sea,6,All day,2,0
26,Cherry salmon,,1000,River (Clifftop),3,4 PM - 9 AM,3,0
27,Char,,3800,River (Clifftop) Pond,3,4 PM - 9 AM,3,0
13,Tadpole,,100,Pond,1,All day,4,0
9,Killifish,,300,Pond,1,All day,5,0


In [35]:
bug_url = "/".join([wiki_base_url, "wiki/Bugs_(New_Horizons)"])

bug_lists = {key: pd.DataFrame(value) for key, value in get_critter_list(bug_url).items()}
bug_list = bug_lists[hemisphere]
bug_hot_list = compute_hot_list(bug_list)
HTML(bug_hot_list.to_html(
    render_links=True,
    escape=False
))

,name,image,price,location,shadow_size,time,months_left,months_until
37,Mole cricket,,500.0,Underground,All day,✓,1,0
1,Common butterfly,,160.0,Flying,4 AM - 7 PM,✓,2,0
2,Yellow butterfly,,160.0,Flying,4 AM - 7 PM,-,2,0
4,Peacock butterfly,,2500.0,Flying by Hybrid Flowers,4 AM - 7 PM,-,2,0
77,Centipede,,300.0,Hitting Rocks,4 PM - 11 PM,✓,2,0
76,Pill bug,,250.0,Hitting Rocks,11 PM - 4 PM,✓,2,0
43,Ladybug,,200.0,On Flowers,8 AM - 5 PM,-,2,0
46,Violin beetle,,450.0,On Tree Stumps,All day,-,2,0
0,Common butterfly,,NaN,160,Flying,4 AM - 7 PM,3,0
24,Honeybee,,200.0,Flying,8 AM - 5 PM,-,3,0
